In [1]:
import requests
from lxml import html
from bs4 import BeautifulSoup

In [23]:
payload = {
    "tel": 4128863483,
    "password": "Sureshalka52#"
}

In [24]:
session_requests = requests.session()

In [25]:
login_url = "https://www.lycamobile.us/en/"

In [26]:
result = session_requests.post(
	login_url, 
	data = payload, 
	headers = dict(referer=login_url)
)

In [42]:
url = "https://www.lycamobile.us/en/my-account/callhistory/"
headers = {'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36"}
cookies = {'SSESSae3ec1328c9a4fa29e363ebd2efb4219':'a25ef627ffecf1e4a91134a170eacabf', '_ga':'GA1.2.503814443.1557873853',
          "_gid":"GA1.2.994879259.1557873853","_fbp":"fb.1.1557873853234.424123947","_icl_current_language":"en","cookie_notice_accepted":"true"}

s = requests.Session()
r = s.get(url, headers=headers, cookies=cookies)

In [105]:
from selenium import webdriver
browser = webdriver.PhantomJS(executable_path="/Users/srossgupta/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs")
browser.set_window_size(1124, 850)


/Users/srossgupta/anaconda3/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [106]:
# my_url = "http://avi.im/stuff/js-or-no-js.html"
browser.get(login_url)
# p_element = browser.find_element_by_id(id_='intro-text')
# print(p_element.text)


In [110]:
button  = browser.find_element_by_id("logginedUserButton")
button.click()

In [112]:
button  = browser.find_element_by_id("login_link")
button.click()

In [114]:
tel_no = browser.find_element_by_id("lyca_login_mobile_no")
password = browser.find_element_by_id("lyca_login_password")

In [115]:
tel_no.send_keys("4128863483")
password.send_keys("Sureshalka52#")

In [117]:
button  = browser.find_element_by_id("login_submit")
button.click()

In [120]:
output = browser.find_elements_by_id("result-set")

In [128]:
url = browser.command_executor._url       #"http://127.0.0.1:60622/hub"
session_id = browser.session_id

In [129]:
driver = webdriver.Remote(command_executor=url,desired_capabilities={})
driver.session_id = session_id

In [165]:
driver.get("https://www.lycamobile.us/en/my-account/callhistory/")

In [166]:
data1={"call_type":[],"dialled_no":[],"date":[],"time":[],"duration":[],"cost":[]}
    
    

In [177]:
table = driver.find_element_by_xpath("//table[@class='table-for-call-history']")

for table_row in table.find_elements_by_xpath(".//tr"):
    items_val = table_row.find_elements_by_xpath(".//td")
#     for row in table_row.find_elements_by_xpath(".//td"):
#         print(row.text)
    print(items_val[0].text)
    data1["call_type"].append(items_val[0].text)
    data1["dialled_no"].append(items_val[1].text)
    data1["date"].append(items_val[2].text)
    data1["time"].append(items_val[3].text)
    data1["duration"].append(items_val[4].text)
    data1["cost"].append(items_val[5].text)
#     print("*"*10)


IndexError: list index out of range

In [176]:
data1["call_type"].append("abc")

In [140]:
values

<selenium.webdriver.remote.webelement.WebElement (session="18dd8250-76bc-11e9-87ec-9d2186aa6f68", element=":wdc:1557891766218")>

In [135]:
table_id = driver.find_element(by="id",value='result-set')


In [137]:
rows = table_id.find_elements(by="TAG_NAME", value="tr")

NoSuchElementException: Message: {"request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Content-Length":"117","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:57387","User-Agent":"selenium/3.141.0 (python mac)"},"httpVersion":"1.1","method":"POST","post":"{\"using\": \"TAG_NAME\", \"value\": \"tr\", \"id\": \":wdc:1557891766217\", \"sessionId\": \"18dd8250-76bc-11e9-87ec-9d2186aa6f68\"}","url":"/elements","urlParsed":{"anchor":"","query":"","file":"elements","directory":"/","path":"/elements","relative":"/elements","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/elements","queryKey":{},"chunks":["elements"]},"urlOriginal":"/session/18dd8250-76bc-11e9-87ec-9d2186aa6f68/element/:wdc:1557891766217/elements"}}
Screenshot: available via screen


In [168]:
driver.save_screenshot('screenshot.png')

True

In [98]:
browser.save_screenshot('screenshot.png')

True

In [28]:
url = 'https://www.lycamobile.us/en/my-account/callhistory/'
result = session_requests.get(
	url, 
	headers = dict(referer = url)
)

In [43]:
soup = BeautifulSoup(r.text,"html.parser")